# Создание агента

:::note

В разделе затрагиваются слдующие основные понятия:

- [Чат-модели](/docs/concepts/#chat-models)
- [Инструменты](/docs/concepts/#tools)
- [Агенты](/docs/concepts/#agents)

:::

Языковые модели просто генерируют текст и не могут выполнять действия.
Разработка *агентов* — это один из основных сценариев использования GigaChain.

Агенты — системы, использующие LLM для рассуждений и определения действий, которые нужно предпринять, а также входных данных, которые нужно при этом использовать.

Результаты этих действий затем могут быть переданы обратно агенту, чтобы он определил, нужно ли делать что-то еще или можно закончить работу.

В разделе приведен пример агента, который может взаимодействовать с несколькими инструментами: локальной базой данных и поисковой системой.
С агентом можно вести разговор и наблюдать как он вызывает инструменты.

## Пример агента

Пример ниже содержит код работающего агента, который использует модель для определения того, какой инструмент нужно вызвать.
Агент подготовлен для работы с поисковым инструментом и обладает разговорной памятью, что позволяет использвоать его в роли развитого чат-бота.

Ниже в разделе приводится пошаговый разбор каждого из компонтов агента.

In [37]:
# Импортирование необходимой функциональности
from langchain_community.chat_models.gigachat import GigaChat
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Создание агента
memory = MemorySaver()
model = GigaChat(
    credentials="авторизационные_данные",
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
    verify_ssl_certs=False,
)
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Использование агента
config = {"configurable": {"thread_id": "abc100"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Привет! Меня зову Вася. Я живу в Москве")]},
    config,
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Узнай погоду в моем городе")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Здравствуйте, Вася! Рада слышать вас. Расскажите, могу ли я вам чем-нибудь помочь?', response_metadata={'token_usage': Usage(prompt_tokens=97, completion_tokens=29, total_tokens=126), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'stop'}, id='run-aad23d97-0cb5-4977-8d2e-b1ce06751157-0')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'погода в москве'}}}, response_metadata={'token_usage': Usage(prompt_tokens=140, completion_tokens=23, total_tokens=163), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'function_call'}, id='run-8989a123-6824-4ae2-839b-77e0290aa584-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'погода в москве'}, 'id': 'fb19fd4f-3bc1-4b7f-9cca-e669ab8d96e4', 'type': 'tool_call'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.gismeteo.ru/weather-moscow-43

:::note

Авторизационные данные — строка, полученная в результате кодирования в Base64 клиентского идентификатора (Client ID) и ключа (Client Secret) API. Вы можете использовать готовые данные из личного кабинета или самостоятельно закодировать идентификатор и ключ.

Пример строки авторизационных данных:

```text
MjIzODA0YTktMDU3OC00MTZmLWI4MWYtYzUwNjg3Njk4MzMzOjljMTI2MGQyLTFkNTEtNGRkOS05ZGVhLTBhNjAzZTdjZjQ3Mw==
```

Идентификатор, ключ и авторизационные данные вы можете получить после создания проекта GigaChat API:

* [для физических лиц](https://developers.sber.ru/docs/ru/gigachat/individuals-quickstart#shag-1-sozdayte-proekt-giga-chat-api);
* [для ИП и юридических лиц](https://developers.sber.ru/docs/ru/gigachat/legal-quickstart#shag-1-otpravte-zayavku-na-dostup-k-proektu-giga-chat-api).

:::

## Подготовка к разработке

### Jupyter-блокноты

Это руководство, как и большинство других в документации, использует [Jupyter-блокноты](https://jupyter.org/). Они отлично подходят для изучения работы с LLM-системами, так как предоставляют интерактивную среду для работы с руководствами и позволяют работать с непредвиденными ситуациями: недоступностью API, нетипичным выводом и другими.

Подробнее об установке jupyter -- в [официальной документации](https://jupyter.org/install).

### Установка

Для установки GigaChain выполните команды:

In [15]:
%%capture --no-stderr
%pip install -U gigachain-community gigagraph tavily-python

Подробнее об установке — в разделе [Установка](https://developers.sber.ru/docs/ru/gigachain/get-started/installation).

<!--
### LangSmith

Многие приложения, которые вы создаете с помощью LangChain, будут содержать несколько шагов с многократными вызовами LLM.
По мере усложнения этих приложений становится важно иметь возможность инспектировать, что именно происходит внутри вашей цепочки или агента.
Лучший способ сделать это — с помощью [LangSmith](https://smith.langchain.com).

После регистрации по ссылке выше, убедитесь, что вы установили переменные среды для начала ведения журнала трассировок:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Или, если вы работаете в блокноте, вы можете установить их с помощью:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```
-->

### Tavily

В качестве инструмента в примере используется поисковый движок [Tavily](/docs/integrations/tools/tavily_search).
Для работы с ним вам нужно получить и задать ключ API:

```bash
export TAVILY_API_KEY="..."
```

В Jupyter-блокноте его можно использовать так:

In [ ]:
# Используйте эту ячейку, чтобы задать ключ в блокноте
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

## Определение инструментов

Сначала создайте инструменты, которые будет использовать агент.
Основным инструментом в примере выступает поисковый движок [Tavily](/docs/integrations/tools/tavily_search).
GigaChain предоставляет интеграцию для простого использования поисковой системы Tavily в качестве инструмента.

In [16]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("Узнай на weatherapi погоду в Москве")
print(search_results)
# Если нужно, вы можете создать другие инструменты.
# После создания всех необходимых инструментов
# их можно сохранить в списке, к которому можно обращаться позднее.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Moscow', 'region': 'Moscow City', 'country': 'Russia', 'lat': 55.75, 'lon': 37.62, 'tz_id': 'Europe/Moscow', 'localtime_epoch': 1726587484, 'localtime': '2024-09-17 18:38'}, 'current': {'last_updated_epoch': 1726587000, 'last_updated': '2024-09-17 18:30', 'temp_c': 22.3, 'temp_f': 72.1, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 4.5, 'wind_kph': 7.2, 'wind_degree': 112, 'wind_dir': 'ESE', 'pressure_mb': 1032.0, 'pressure_in': 30.47, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 41, 'cloud': 0, 'feelslike_c': 23.8, 'feelslike_f': 74.8, 'windchill_c': 22.0, 'windchill_f': 71.5, 'heatindex_c': 23.7, 'heatindex_f': 74.7, 'dewpoint_c': 5.9, 'dewpoint_f': 42.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 6.0, 'gust_mph': 8.4, 'gust_kph': 13.5}}"}, {'url': 'https://yandex.ru/pogoda/moscow/month/september', 'content': 'Погода в Москве в 

## Использование языковых моделей

Пример ниже показвыает как исползовать языковую модель для вызова инструментов.
Кроме моделей GigaChat, GigaChain позволяет использовать и другие модели.

In [19]:
# | output: false
# | echo: false

from langchain_community.chat_models.gigachat import GigaChat

model = GigaChat(
    credentials="авторизационные_данные",
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
    verify_ssl_certs=False,
)

Для вызова модели передайте ей список сообщений.
По умолчанию ответом будет строка `content`.

In [20]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="Привет!")])
response.content

'Здравствуйте! Я могу вам чем-нибудь помочь?'

Используйте метод `.bind_tools`, чтобы дать модели знать к каким инструментам она может обращаться.

In [21]:
model_with_tools = model.bind_tools(tools)

Сначала, посмотрите как ответит модель на обчное сообщение.
Ответ модели можно найти в двух полях `content` и `tool_calls`.

In [22]:
response = model_with_tools.invoke([HumanMessage(content="Привет!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Здравствуйте! Я могу вам чем-нибудь помочь?
ToolCalls: []


Теперь попробуйте вызвать модель с помощью запроса, который предполагает использование инструмента.

In [23]:
response = model_with_tools.invoke(
    [HumanMessage(content="Какая погода в Москве, спроси у weatherapi?")]
)

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Какая погода в Москве, спроси у weatherapi?'}, 'id': 'c3b8aae6-8fec-4516-9c5b-c43e4465669c', 'type': 'tool_call'}]


Поле ContentString теперь пустое, но по содержимому массива ToolCalls видно, что модель хочет, чтобы вы вызвали инструмент поиска Tavily.

Модель не может самостоятельно вызвать инструмент, для этого нужно создать агента.

## Создание агента

Для создания агента используйте [GigaGraph](/docs/concepts/#langgraph) — инструмент, который предоставляет высокоуровневый интерфейс для создания агентов, а также дает доступ к низкоуровневым инструментам, которые дают полный контроль над логикой работы агента.

Инициализируйте агент с моделью и набором инструментов.

:::note

В агент можно передавать `model`, а не `model_with_tools`, т.к. метод `create_react_agent` самостоятельно вызывает `.bind_tools`.

:::

In [24]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Запуск агента

Теперь вы можете проверить работу агента на нескольких запросах.

Сейчас агент не сохраняет информацию о состоянии и не помнит историю взаимодействия с пользователем.

В результате работы агент возвращает итоговое состояние, которое кроме выходных данных также содержит все входные данные.
Как получить только выходные данные показано в примерах ниже.

Посмотрите как агент отвечает, когда не нужно вызывать инструменты:

In [25]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Привет!")]})

response["messages"]

[HumanMessage(content='Привет!', id='039c0ab3-0e7e-4544-afca-0a5041d20dd0'),
 AIMessage(content='Здравствуйте! Я могу вам чем-нибудь помочь?', response_metadata={'token_usage': Usage(prompt_tokens=86, completion_tokens=16, total_tokens=102), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'stop'}, id='run-59315bc5-35a2-4ba4-b4ba-5a02aee9d375-0')]

<!--
In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)
-->

Попробуйте обратиться к агенту с запросом, который предполагает вызов инструмента:

In [26]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Какая погода в Москве, узнай на weatherapi?")]}
)
response["messages"]

[HumanMessage(content='Какая погода в Москве, узнай на weatherapi?', id='745327d6-f6ce-4664-ab06-c49f517c4835'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'какая погода в москве, узнать на weatherapi?'}}}, response_metadata={'token_usage': Usage(prompt_tokens=96, completion_tokens=31, total_tokens=127), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'function_call'}, id='run-374b7c90-7807-469b-9d05-100b202e885e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'какая погода в москве, узнать на weatherapi?'}, 'id': 'e279257d-01ff-4886-bff1-6861f4645331', 'type': 'tool_call'}]),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'\\u041c\\u043e\\u0441\\u043a\\u0432\\u0430\', \'region\': \'Moscow City\', \'country\': \'\\u0420\\u043e\\u0441\\u0441\\u0438\\u044f\', \'lat\': 55.75, \'lon\': 37.62, \'tz_id\': \'Europe/Moscow\', \'loc

<!--
We can check out the [LangSmith trace](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) to make sure it's calling the search tool effectively.
-->

## Потоковая передача сообщений

В предыдущих примерах агент вызывается с помощью метода `.invoke` и возвращает итоговой ответ.
Это может занимать продолжительное время, если агент в процессе работы выполняет множество шагов.
Для демонстрации промежуточных результатов вы можете передавать сообщения по мере их возникновения.

In [27]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Какая погода в Москве, узнай на weatherapi?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'какая погода в москве, узнать на weatherapi?'}}}, response_metadata={'token_usage': Usage(prompt_tokens=96, completion_tokens=31, total_tokens=127), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'function_call'}, id='run-cbd1e6f5-d99d-46bb-a54f-1482de12a277-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'какая погода в москве, узнать на weatherapi?'}, 'id': '414756b7-d87a-46e3-83b4-e4f688efff58', 'type': 'tool_call'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'\\u041c\\u043e\\u0441\\u043a\\u0432\\u0430\', \'region\': \'Moscow City\', \'country\': \'\\u0420\\u043e\\u0441\\u0441\\u0438\\u044f\', \'lat\': 55.75, \'lon\': 37.62, \'tz_id\': \'Europe/Moscow\', \'localtime_epoch\': 1726587954, \'localtime\': \'2024-09-17 18:4

## Потоковая передача токенов

Кроме сообщений вы также можете использовать потоковую передачу токенов.
Для этого используйте метод `.astream_events`.

:::important

Метод `.astream_events` работает в версиях Python 3.11 и выше.

:::

In [28]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="Какая погода в Москве, узнай на weatherapi?")]},
    version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Назначается при создании агента с помощью метода `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Назначается при создании агента с помощью метода `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

<ipython-input-28-6a1dd78ae5f1>:1: LangChainBetaWarning: This API is in beta and may change in the future.
  async for event in agent_executor.astream_events(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'какая погода в москве, узнать на weatherapi?'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Moscow\', \'region\': \'Moscow City\', \'country\': \'Russia\', \'lat\': 55.75, \'lon\': 37.62, \'tz_id\': \'Europe/Moscow\', \'localtime_epoch\': 1726588046, \'localtime\': \'2024-09-17 18:47\'}, \'current\': {\'last_updated_epoch\': 1726587900, \'last_updated\': \'2024-09-17 18:45\', \'temp_c\': 21.2, \'temp_f\': 70.2, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 4.5, \'wind_kph\': 7.2, \'wind_degree\': 112, \'wind_dir\': \'ESE\', \'pressure_mb\': 1032.0, \'pressure_in\': 30.47, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 43, \'cloud\': 0, \'feelslike_c\': 21.2, \'feelslike_f\': 70.2, \'windchill_c\': 22.0, \'windchill

## Добавление памяти

Для добавления памяти нужно при вызове агента передать ему чекпойнтер и поле `thread_id`.
С помощью этог поля агент сможет понять к какому разговору нужно вернуться.

In [29]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [30]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [31]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Привет! Меня зовут Вася")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Здравствуйте, Вася! Рада слышать вас.', response_metadata={'token_usage': Usage(prompt_tokens=92, completion_tokens=17, total_tokens=109), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'stop'}, id='run-c2c24921-b788-4196-8311-4cf4ffdc0352-0')]}}
----


In [32]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Как меня зовут?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Вас зовут Вася.', response_metadata={'token_usage': Usage(prompt_tokens=121, completion_tokens=9, total_tokens=130), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'stop'}, id='run-71dc1dce-8219-4c1d-b2ba-d9bcb008af11-0')]}}
----


<!--
Example [LangSmith trace](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)
-->

Для начала нового разговора вам достаточно изменить значение `thread_id`.

In [33]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Как меня зовут?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Я генеративная языковая модель и не имею доступа к вашим персональным данным. Предполагаю, что вас зовут Иван. Правильно ли я предположила?', response_metadata={'token_usage': Usage(prompt_tokens=89, completion_tokens=39, total_tokens=128), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'stop'}, id='run-3695cc4f-4007-4fb5-9b25-c5cb5076aa6e-0')]}}
----


## Смотрите также

Более подробную информацию о разработке агентов ищите в документации [GigaGraph](/docs/concepts/#langgraph).